# Let's use shor's algorithm to factorize the number 65

In [ ]:
!pip install qiskit --upgrade
!pip install pylatexenc
!pip install scipy --upgrade

In [ ]:
import qiskit
from qiskit import IBMQ
from qiskit import *
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from qiskit.quantum_info import Statevector, partial_trace
from qiskit.quantum_info.operators import Operator
%matplotlib inline
from sympy import Symbol, Matrix, eye, exp, conjugate, sqrt, pi
from sympy.physics.quantum import TensorProduct
import numpy as np
from numpy import pi as np_pi
import math
import random
from fractions import Fraction
#Auxiliar functions
def local_simulation(qc, shots=1000):
    simulator = Aer.get_backend('qasm_simulator')
    result = execute(qc, simulator, shots=shots, memory=True).result()
    values = result.get_counts(qc)
    return values, result

### We first build the order finding quantum subroutine

The Unitary matrix to be used will be determined 
- u starts with state([0, 1]) because |1> is a superposition of the eigenvalues of U 
- In the second register we'll have log2(N) qubits

In [ ]:
def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ"""
    qc = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

def get_U(a, N, second_register_len):
  maxN = 2**second_register_len; # Max number represented using second_register_len bits
  U = [];
  for i in range(N):# i takes values 0 to N-1
      aux = a*i % N
      st = np.zeros(maxN)
      st[aux] = 1
      U.append(st);

  for i in range(N, maxN):# i takes values N to maxN
      st = np.zeros(maxN)
      st[i] = 1
      U.append(st);
  U = np.transpose(np.array(U))
  print(f"Built U with shape: {U.shape}")
  return U

def build_QPE_circuit(a, N=65, t=8):
  second_register_len = math.ceil(math.log2(N))
  U = get_U(a, N, second_register_len)# Building U
  qc_phase_estimation = QuantumCircuit(QuantumRegister(t + second_register_len), ClassicalRegister(t), name="QPE")
  
  # STEP 0: Initialize. Second register qubits take the value 1
  #qc_phase_estimation.x(t) #
  #qc_phase_estimation.x(range(t, t + second_register_len))
  qc_phase_estimation.barrier()

  # STEP 1: H on first register
  qc_phase_estimation.h(range(t))
  qc_phase_estimation.barrier()
  
  # STEP 2: appplying unitary matrix
  for i in range(1, t+1):
    #print(f"Applying unitary 2^{i} for a={a}")
    circuit = QuantumCircuit(QuantumRegister(second_register_len), name=f'2^{i}')
    circuit.unitary(Operator(U**(2**(i-1))), range(second_register_len))
    custom_gate = circuit.to_gate().control()
    control_qubit_index = t - i
    qubits = [control_qubit_index] + list(range(t, t + second_register_len))
    qc_phase_estimation.append(custom_gate, qubits)
  
  qc_phase_estimation.barrier()
  
  # STEP 3: Applying inverse QFT
  qc_phase_estimation.append(qft_dagger(t), range(t))# QFT with t qubits
  qc_phase_estimation.barrier()
  
  # Adding measurements
  qc_phase_estimation.measure(range(t), range(t))
  
  return qc_phase_estimation, U

def get_phase(a, N, t):
    print(f"Getting Phase for a={a}, N={N}, t={t}")
    qc, U = build_QPE_circuit(a, N, t)
    # Simulate Results
    _, result = local_simulation(qc)
    readings = result.get_memory()
    print("Register Reading: " + readings[0])
    phase = int(readings[0], 2)/(2**t)
    print("Corresponding Phase: %f" % phase)
    frac = Fraction(phase).limit_denominator(N) # Denominator should (hopefully!) tell us r
    r = frac.denominator
    return phase, r

### Now let's get to Shor's algorithm

In [ ]:
def get_ab_factors(N):
  alpha = math.log2(N)
  L = math.ceil(alpha) 
  print(f"alpha: {alpha}, L: {L}")
  for b in range(2, L):
    x = alpha/b
    u = 2**x
    u1 = math.floor(u)
    u2 = math.ceil(u)
    if u1**b == N:
      return u1, b
    if u2**b == N:
      return u2, b

  return None, None

In [ ]:
def get_shor_factorization(N, t=8):
  # Step 1: If N is even, return the factor of 2
  if N % 2 == 0:
    return 2, N//2
  
  # Step 2:  Determine whether N = a^b for integers a > 1 and b =>2, and if so return the factor a.
  a, b = get_ab_factors(N)
  if a is not None:
    print(f"a: {a}, b: {b}")
    return a, N//a

  k_values = list(range(2, N))
  random.shuffle(k_values)
  for k in k_values:
    print("\n")
    # Step 3: Randomly choose k in the range 2 to N-1. If gcd(k, N) > 1 then return the factor gcd(k, N)
    gcd_k_n = math.gcd(k, N)
    if gcd_k_n > 1:
      print(f"gcd({k}, {N}): {gcd_k_n}")
      return gcd_k_n, N//gcd_k_n
  
    # Step 4: Use the order finding quantum subroutine to find the order r of k (modulo N)
    # TODO probar si obtener la phase varias veces para el mismo k ayuda
    phase, r = get_phase(k, N, t)
    print(f"Obtained factor r={r} for k={k}")
    # Step 5: If r is even and k^(r/2) != -1 mod N then compute gcd(K^(r/2)-1, N) and gcd(K^(r/2)+1, N), and test to see if one of these is a non trivial factor. If it fails return to step 3
    if phase > 0: #(r % 2 == 1) or (k**(r//2) == (-1 % N) ):
      gcd_lower = math.gcd(k**(r//2) - 1, N)
      gcd_upper = math.gcd(k**(r//2) + 1, N)
      print(f"Evaluating Factors: {gcd_lower} and {gcd_upper}")
      for guess in [gcd_lower, gcd_upper]:
          if guess not in [1,N] and (N % guess) == 0: # Check to see if guess is a factor
              print(f"Non-trivial factor found: {guess}")
              factor_found = True
              return guess, N//guess
    
  return None, None

In [ ]:
# con 1000.
a, b = get_shor_factorization(N=15, t=3)
a, b

alpha: 3.9068905956085187, L: 4


gcd(3, 15): 3


(3, 5)

In [ ]:
# con 1111..
a, b = get_shor_factorization(N=15, t=3)
a, b

alpha: 3.9068905956085187, L: 4


Getting Phase for a=11, N=15, t=3
Built U with shape: (16, 16)
Register Reading: 000
Corresponding Phase: 0.000000
Obtained factor r=1 for k=11


Getting Phase for a=7, N=15, t=3
Built U with shape: (16, 16)
Register Reading: 000
Corresponding Phase: 0.000000
Obtained factor r=1 for k=7


gcd(10, 15): 5


(5, 3)

In [ ]:
a, b = get_shor_factorization(N=65, t=3)
a, b

alpha: 6.022367813028454, L: 7


Getting Phase for a=61, N=65, t=3
Built U with shape: (128, 128)
Register Reading: 101
Corresponding Phase: 0.625000
Obtained factor r=8 for k=61
Evaluating Factors: 5 and 1
Non-trivial factor found: 5


(5, 13)

### The built shor's algorithm outputs correctly 3 and 15 as the factors of 65